# Monte Carlo Method
## Single Parameter

In [13]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

def compute_err(U, U_exact):
    n = len(U)
    err_inf = 0
    err_sq = 0
    for i in range(0,n):
        for j in range(0,n):
            err_sq += np.absolute(U_exact[i][j] - U[i][j])**2
            if np.absolute(U_exact[i][j] - U[i][j]) > err_inf:
                err_inf = np.absolute(U_exact[i][j] - U[i][j])
        
    err_sq = (err_sq * h**2)**0.5
    return err_inf, err_sq

def sim_trans_explicit(T, F):
    start_time = time.time()
    eigvals = np.empty(n)
    eigvecs = np.empty([n,n])
    i = np.linspace(0,n-1,n)
    j = np.linspace(0,n-1,n)
    I, J = np.meshgrid(i, j, indexing='ij')
    eigvals = (T[0][0] - T[0][0]*np.cos((i+1)*np.pi/(n+1)))
    eigvecs = np.sqrt(2/(n+1))*np.sin(((I+1)*(J+1)*np.pi)/(n+1))
    eig_time = time.time() - start_time
    diag = diags(eigvals, 0, shape=(n, n)).toarray() #Set eigenvalues as diagonal matrix
    P = eigvecs #Set basis matrix P, with columns as the eigenvectors
    P_trans = P.transpose()
    diag_time = time.time() - eig_time - start_time
    #F_hat = np.matmul(np.matmul(np.linalg.inv(P),F),P) #F_hat = P^-1 * F * P
    F_hat = np.matmul(np.matmul(P_trans,F),P) #F_hat = P^-1 * F * P
    F_time = time.time() - diag_time - eig_time - start_time

    U_hat = np.empty([n,n]) #u_hat(i,j) = f(i,j) / eigval(i) + eigval(j)
    for i in range(0,n):
        for j in range(0,n):
            if (eigvals[i]!=0 or eigvals[j]!=0):
                U_hat[i][j] = F_hat[i][j] / (eigvals[i]+eigvals[j])
            else:
                U_hat[i][j] = 0

    U_time = time.time() - F_time - diag_time - eig_time - start_time
    #U = np.matmul(np.matmul(P,U_hat),np.linalg.inv(P)) #U = P * U^hat * P^-1
    U = np.matmul(np.matmul(P,U_hat),P_trans) #U = P * U^hat * P^-1
    end_time = time.time()
    solve_time = end_time - U_time - F_time - diag_time - eig_time - start_time
    total_time = end_time - start_time
    timings = [eig_time, diag_time, F_time, U_time, solve_time, total_time]
    return U, timings

In [16]:
import numpy as np
from scipy.sparse import diags
import random
import time

def monte_carlo(r1, r2, M):
    
    random.seed(0)

    #Define mean, variance and solutions as zero matrices
    mean_sol = np.zeros([n,n])
    #var_sol = np.zeros([n,n])
    
    
    #sols = np.zeros([M,n,n])
    
    start_time = time.time()
        
    for i in range(0, M): 
        eps = np.random.uniform(r1, r2) #Generate random eps in range(r1,r2)
        T = np.multiply((-eps)/(h**2), diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())
        F = (2*(np.pi**2)*eps*np.sin(np.pi*X)*np.sin(np.pi*Y))+(34*(np.pi**2)*(eps**2)*np.sin(3*np.pi*X)*np.sin(5*np.pi*Y))
        U = sim_trans_explicit(T,F)[0]
        #sols[i] = U
        mean_sol += U
        
    mean_sol = mean_sol/M #Divide by number of samples to get the mean solution
    
    total_time = time.time() - start_time
    
    return mean_sol, total_time
#     plot_sol(X,Y,mean_sol)
    
#     #Calculate variance
#     for i in range(0, M):
#         var_sol += ((sols[i] - mean_sol)**2)
#         var_sol = var_sol/(M-1)
    








In [19]:
import numpy as np
from scipy.sparse import diags
import sys

#Define parameters
n = 125
h = 1/(n+1) #step size

#Define x and y as arrays between 0 and 1 with n evenly spaced points (internal nodes)
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Create internal mesh (excludes boundaries)
X, Y = np.meshgrid(x, y, indexing='ij')

#Define tridiagonal matrix T
diagonals = [[-2],[1],[1]]

#Compute exact solution for comparison
U_exact = np.sin(np.pi*X) * np.sin(np.pi*Y) + 2*np.sin(3*np.pi*X)*np.sin(5*np.pi*Y)

U, total_time = monte_carlo(1,3,10)

err_inf, err_sq = compute_err(U, U_exact)
print('Time taken:', total_time)
print('Error inf:', err_inf, '\nError sq:' ,err_sq)



Time taken: 2.7188472747802734
Error inf: 0.09895283365980401 
Error sq: 0.049463471789217735


### For SSH

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import diags
import random
import time
import sys

def compute_err(U, U_exact):
    n = len(U)
    err_inf = 0
    err_sq = 0
    for i in range(0,n):
        for j in range(0,n):
            err_sq += np.absolute(U_exact[i][j] - U[i][j])**2
            if np.absolute(U_exact[i][j] - U[i][j]) > err_inf:
                err_inf = np.absolute(U_exact[i][j] - U[i][j])
        
    err_sq = (err_sq * h**2)**0.5
    return err_inf, err_sq

def sim_trans_explicit(T, F):
    start_time = time.time()
    eigvals = np.empty(n)
    eigvecs = np.empty([n,n])
    i = np.linspace(0,n-1,n)
    j = np.linspace(0,n-1,n)
    I, J = np.meshgrid(i, j, indexing='ij')
    eigvals = (T[0][0] - T[0][0]*np.cos((i+1)*np.pi/(n+1)))
    eigvecs = np.sqrt(2/(n+1))*np.sin(((I+1)*(J+1)*np.pi)/(n+1))
    eig_time = time.time() - start_time
    diag = diags(eigvals, 0, shape=(n, n)).toarray() #Set eigenvalues as diagonal matrix
    P = eigvecs #Set basis matrix P, with columns as the eigenvectors
    P_trans = P.transpose()
    diag_time = time.time() - eig_time - start_time
    #F_hat = np.matmul(np.matmul(np.linalg.inv(P),F),P) #F_hat = P^-1 * F * P
    F_hat = np.matmul(np.matmul(P_trans,F),P) #F_hat = P^-1 * F * P
    F_time = time.time() - diag_time - eig_time - start_time

    U_hat = np.empty([n,n]) #u_hat(i,j) = f(i,j) / eigval(i) + eigval(j)
    for i in range(0,n):
        for j in range(0,n):
            if (eigvals[i]!=0 or eigvals[j]!=0):
                U_hat[i][j] = F_hat[i][j] / (eigvals[i]+eigvals[j])
            else:
                U_hat[i][j] = 0

    U_time = time.time() - F_time - diag_time - eig_time - start_time
    #U = np.matmul(np.matmul(P,U_hat),np.linalg.inv(P)) #U = P * U^hat * P^-1
    U = np.matmul(np.matmul(P,U_hat),P_trans) #U = P * U^hat * P^-1
    end_time = time.time()
    solve_time = end_time - U_time - F_time - diag_time - eig_time - start_time
    total_time = end_time - start_time
    timings = [eig_time, diag_time, F_time, U_time, solve_time, total_time]
    return U, timings

def monte_carlo(r1, r2, M):
    
    random.seed(0)
    
    #Define mean as zero matrix
    mean_sol = np.zeros([n,n])
        
    start_time = time.time()
        
    for i in range(0, M): 
        eps = random.uniform(r1, r2) #Generate random eps in range(r1,r2)
        T = np.multiply((-eps)/(h**2), diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())
        F = (2*(np.pi**2)*eps*np.sin(np.pi*X)*np.sin(np.pi*Y))+(34*(np.pi**2)*(eps**2)*np.sin(3*np.pi*X)*np.sin(5*np.pi*Y))
        U = sim_trans_explicit(T, F)[0]
        #sols[i] = U
        mean_sol += U
        
    mean_sol = mean_sol/M #Divide by number of samples to get the mean solution
    
    total_time = time.time() - start_time
    
    return mean_sol, total_time

#Define parameters
n = int(sys.argv[1])
h = 1/(n+1) #step size

#Define x and y as arrays between 0 and 1 with n evenly spaced points (internal nodes)
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Create internal mesh (excludes boundaries)
X, Y = np.meshgrid(x, y, indexing='ij')

#Define tridiagonal matrix T
diagonals = [[-2],[1],[1]]

#Compute exact solution for comparison
U_exact = np.sin(np.pi*X) * np.sin(np.pi*Y) + 2*np.sin(3*np.pi*X)*np.sin(5*np.pi*Y)

M = int(sys.argv[2])

U, total_time = monte_carlo(1,3,M)

err_inf, err_sq = compute_err(U, U_exact)
print('Time taken:', total_time)
print('Error inf:', err_inf, '\nError sq:' ,err_sq)

    






